In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from semiZ import fitSemiempirical, calcLookupTables, Lookup, calcZ

# Step 1:  Load data

### Load the beam spectra and detector response

In [ ]:
phi_H = np.load("data/phi_9MeV.npy")
phi_L = np.load("data/phi_6MeV.npy")
D = np.load("data/D.npy")
E = np.load("data/E.npy")

In [ ]:
plt.figure(figsize = [8, 6])
plt.plot(E, phi_H[0], color='r', linewidth=3, label = r"9 MeV spectrum at $\theta = 0^\circ$")
plt.plot(E, phi_L[0], color='b', linewidth=3, label = r"6 MeV spectrum at $\theta = 0^\circ$")
plt.xlabel("Energy (MeV)", fontsize=19)
plt.ylabel("Differential Flux (MeV$^{-1}$)", fontsize=19)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid()
plt.xlim([0, 9])
plt.gca().set_ylim(bottom=0)
plt.legend(fontsize=19)

### Load calibration data

In [ ]:
alpha_H_calib = np.load("data/alpha_H_calib.npy")
alpha_L_calib = np.load("data/alpha_L_calib.npy")
lmbda_calib = np.load("data/lmbda_calib.npy")
Z_calib = np.load("data/Z_calib.npy")

# Step 2: Perform the calibration

In [ ]:
k_bins = phi_H.shape[0]

a_H = np.zeros(k_bins)
b_H = np.zeros(k_bins)
c_H = np.zeros(k_bins)
a_L = np.zeros(k_bins)
b_L = np.zeros(k_bins)
c_L = np.zeros(k_bins)

for i in range(k_bins):
    a_H[i], b_H[i], c_H[i] = fitSemiempirical(alpha_H_calib[i], lmbda_calib[i], Z_calib[i], phi_H[i], D, E)
    a_L[i], b_L[i], c_L[i] = fitSemiempirical(alpha_L_calib[i], lmbda_calib[i], Z_calib[i], phi_L[i], D, E)

# Step 3: precompute the forward model

### Use calibration parameters to compute lookup tables

In [ ]:
lmbda_range = np.linspace(0, 300, 301)
Z_range = np.arange(1, 101)
tables = calcLookupTables(phi_H, phi_L, D, E, a_H, b_H, c_H, a_L, b_L, c_L, lmbda_range, Z_range)

### Use the lookup tables to define a Lookup object for computing the forward model

In [ ]:
lookup = Lookup(tables, lmbda_range, Z_range, interpolate_lmbda = True)

# Step 4:  run analysis on cargo images

### Load simulation results

In [ ]:
im_H = np.load("data/im_H.npy")
im_L = np.load("data/im_L.npy")

plt.figure(figsize = [20, 4])
plt.imshow(im_H, cmap="plasma", extent=[0, im_H.shape[1], 0, im_H.shape[0]], vmin=0, vmax=4, interpolation="nearest")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Horizontal Pixel Index", fontsize=19)
plt.ylabel("Vertical Pixel Index", fontsize=19)
cb = plt.colorbar(pad=0.01)
cb.set_label(r"$\alpha_H$", fontsize=19)
cb.ax.tick_params(labelsize=14)

### Reconstruct $Z$ without segmentation

In [ ]:
lmbda, Z = calcZ(im_H, im_L, lookup)

transparency = np.minimum(lmbda / np.percentile(lmbda, 99), 1)
plt.figure(figsize = [20, 4])
plt.imshow(Z, alpha=transparency, extent=[0, Z.shape[1], 0, Z.shape[0]], 
           vmin=1, vmax=80, interpolation="nearest")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Horizontal Pixel Index", fontsize=19)
plt.ylabel("Vertical Pixel Index", fontsize=19)
cb = plt.colorbar(pad=0.01)
cb.set_label("Atomic Number", fontsize=19)
cb.ax.tick_params(labelsize=14)

### Reconstruct $Z$ with segmentation

In [ ]:
from skimage.segmentation import felzenszwalb
scale = 2000
sigma = 0.0
min_size = 20
labels = felzenszwalb(np.dstack((im_H, im_L)), scale=scale, sigma=sigma, min_size=min_size)

In [ ]:
lmbda, Z = calcZ(im_H, im_L, lookup, labels=labels)

transparency = np.minimum(lmbda / np.percentile(lmbda, 99), 1)
plt.figure(figsize = [20, 4])
plt.imshow(Z, alpha=transparency, extent=[0, Z.shape[1], 0, Z.shape[0]], 
           vmin=1, vmax=80, interpolation="nearest")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Horizontal Pixel Index", fontsize=19)
plt.ylabel("Vertical Pixel Index", fontsize=19)
cb = plt.colorbar(pad=0.01)
cb.set_label("Atomic Number", fontsize=19)
cb.ax.tick_params(labelsize=14)